In [1]:
import numpy as np
import numpy.ma as ma
import nmf as nimfa
import copy
import csv

import sys
sys.path.append("..")
import utils as ut
import STMF_ByMatrix as stmf  # choose method

/home/amra/anaconda3/envs/venv/lib/python3.6/site-packages/nimfa/examples/cbcl_images.py:98: UserWarning: PIL must be installed to run CBCL images example.
  warn("PIL must be installed to run CBCL images example.")
/home/amra/anaconda3/envs/venv/lib/python3.6/site-packages/nimfa/examples/orl_images.py:110: UserWarning: PIL must be installed to run ORL images example.
  warn("PIL must be installed to run ORL images example.")


In [2]:
def create_dataset(lamb, A, B):
    trop_matrix = ut.max_plus(A, B.T)
    stand_matrix = ma.masked_array(np.matmul(A.data, B.T.data), mask=np.zeros((m,n)))
    return (lamb * trop_matrix + (1-lamb)*stand_matrix).astype(np.float32)

In [3]:
lamb = 0.5 # 0 for linear structure, 1 for tropical structure and 0.5 for mixture
transpose = True # True or False, tall or wide matrix
# one run of this file for normal data and one run for transposed data
number_of_datasets = 50
m = 10000  # number of rows, 200 or 10 000
n = 300  # number of columns, 200 or 300
rank = 3  # rank
missing_value = 0  
repeat = 10  # 10
sparsity = 0.2  # 20%
init_tmf = 'random_vcol'
folder = "final_tall_vs_wide/"
version = "bymatrix" #
time_in_seconds = 600 # 100 or 600 
seed_list = np.arange(1, number_of_datasets + 1) * 10 # training datasets
#seed_list = (np.arange(1, number_of_datasets + 1) * 10) + 1  # test datasets

In [4]:
def generate_data(seed):
    np.random.seed(seed)
    A = ma.masked_array(np.random.rand(m, rank), mask=np.zeros((m, rank)))  # uniform distribution over [0, 1)
    B = ma.masked_array(np.random.rand(n, rank), mask=np.zeros((n, rank)))

    X_maxplus_orig = create_dataset(lamb, A, B)
    ut.check_zeros(X_maxplus_orig)
    X_maxplus = copy.deepcopy(X_maxplus_orig)

    X_maxplus = ut.create_matrix_with_missing_values(X_maxplus, sparsity, missing_value)  # create matrix with missing values
    X_missing_values = copy.deepcopy(X_maxplus)
    X_maxplus = ma.masked_equal(X_maxplus, missing_value)  # create masked array
    
    if transpose == True:
        X_maxplus_orig = X_maxplus_orig.T
        X_missing_values = X_missing_values.T
        X_maxplus = X_maxplus.T

    return X_maxplus_orig, X_missing_values, X_maxplus

In [5]:
seed = 42
for s in seed_list: # different datasets
    X_maxplus_orig, X_missing_values, X_maxplus = generate_data(s)

    for i in range(rank, rank+1): # rank=3
        for j in range(0, repeat): # 10x
            model = stmf.STMF(rank=i, initialization=init_tmf, threshold=time_in_seconds, seed_param=seed+s, j_param=j)
            model.fit(X_maxplus)
            model.get_statistics(version, s, j, folder, transpose)